In [ ]:
%%sh
sudo pip install spark
sudo pip install pyspark

In [ ]:
#1-Instalar e importar as bibliotecas do spark e pyspark (as mesmas disponibilizadas nas aulas).
import pandas as pd

#Importando o spark e o pyspark
import spark,pyspark

#Importando as bibliotecas do pyspark.sql 
from pyspark.sql import *

#Importando as funções sql do spark
#documentação https://spark.apache.org/docs/latest/api/sql/index.html
from pyspark.sql import functions as f

#Importando os tipos de dados do spark
#documentação https://spark.apache.org/docs/latest/sql-ref-datatypes.html
from pyspark.sql import types as t 

#Biblioteca datetime
from datetime import datetime, date

In [ ]:
#2 - Criar um spark context com o app name = “AC4 DataEng”.
spark = SparkSession.builder.master("local").appName("AC5 DataEng").getOrCreate()
#3-Subir o arquivo “vendas.parquet” para o ambiente do Colab.
#4-Criar um dataframe lendo o arquivo vendas.parquet.
#5-Converter a coluna data para o tipo DateType.
#6-Criar as colunas, utilizando as funções spark.sql.functions.year,spark.sql.functions.month ,spark.sql.functions.dayofyear:
 #ano = ano da coluna “data”.
 #mes= mes da coluna “data”.
 #dia= dia da coluna “data”.
#7-Na coluna sms, se caso o valor for null, troque por “False” (use a função spark.sql.functions.when)
#8-Crie uma coluna chamada “total” que deve ser a multiplicação das colunas “vlr” e “qtd”.
#9-Grave o dataframe particionado por ano, mes e dia no formato “parquet” de nome “vendas_tratadas”.
#10-Crie uma tabela temporaria chamada “vendas_table” (dataFrame.registerTempTable(“vendas_table”))
#11-Crie um dataframe spark.sql utilizando a query spark.sql(“select distinct cidade from vendas_table”). Grave esse dataframe no formato “parquet” de nome  “cidade”.
#12-Criar um dataframe com a soma do vlr, soma da qtd agrupado pela data. Gravar esse dataframe no formato “parquet” de nome “vendas_total”. 
#13-Crie um novo dataframe chamado “produto” selecionando as colunas: “des_produto”,”des_familia”, “des_secao”, “des_categoria”,”des_sub_categoria”. Apenas valores unicos (distinct()). Grave esse dataframe como parquet “produto”
#14-Crie um dataframe chamado “venda_mulheres”, selecionando todas as colunas e filtrando apenas vendas realizadas para o sexo “F”, grave esse dataframe no formato json “venda_mulher”. (df.write.format(“json”).save(“venda_mulher”))
#15-Crie um dataframe chamado “venda_unicas”, selecionando todas as colunas e filtrando apenas vendas com a qtd igual 1 e o vlr menor que 10. Grave esse dataframe no formato csv“venda_unica”. (df.write.format(“csv”).save(“venda_unica”))


In [ ]:
#4-Criar um dataframe lendo o arquivo vendas.parquet
df = spark.read.format("parquet").load("/content/vendas1.parquet")
df = spark.read.parquet("/content/vendas1.parquet")
 

In [ ]:
#5-Converter a coluna data para o tipo DateType.
df = df.withColumn("data",df.data.cast(t.DateType()))
df = df.withColumn("cod_pessoa",df.cod_pessoa.cast(t.DecimalType(10,4)))
df = df.withColumn("cod_transacao",df.cod_transacao.cast("Decimal(10,2)"))
#6-Criar as colunas, utilizando as funções spark.sql.functions.year,spark.sql.functions.month ,spark.sql.functions.dayofmonth:
df = df.withColumn("ano",f.year(df.data)) 
df = df.withColumn("mes",f.month(df.data)) 
df = df.withColumn("dia",f.dayofmonth(df.data)) 
df.select("data","cod_pessoa","cod_transacao","ano","mes","dia").show()

+----------+----------+-------------+----+---+---+
|      data|cod_pessoa|cod_transacao| ano|mes|dia|
+----------+----------+-------------+----+---+---+
|2020-10-26| 4644.0000|   8064284.00|2020| 10| 26|
|2020-10-26| 3578.0000|   8067745.00|2020| 10| 26|
|2020-12-18| 6505.0000|   8301911.00|2020| 12| 18|
|2020-11-25| 3982.0000|   8194983.00|2020| 11| 25|
|2020-11-05| 1195.0000|   8109606.00|2020| 11|  5|
|2020-11-08|36979.0000|   8123367.00|2020| 11|  8|
|2020-10-17|14173.0000|   8027160.00|2020| 10| 17|
|2020-10-17| 3618.0000|   8025931.00|2020| 10| 17|
|2020-10-17| 4860.0000|   8026361.00|2020| 10| 17|
|2020-12-11|23792.0000|   8270907.00|2020| 12| 11|
|2020-11-06|42057.0000|   8114968.00|2020| 11|  6|
|2020-11-20|12539.0000|   8174079.00|2020| 11| 20|
|2020-11-23|23881.0000|   8188049.00|2020| 11| 23|
|2020-12-03|12539.0000|   8232779.00|2020| 12|  3|
|2020-09-22| 1195.0000|   7917141.00|2020|  9| 22|
|2020-09-22|14173.0000|   7916639.00|2020|  9| 22|
|2020-10-19| 2274.0000|   80365

In [ ]:
#7-Na coluna sms, se caso o valor for null, troque por “False” (use a função spark.sql.functions.when)
df = df.withColumn("sms",f.when(df.sms.isNull(),False).when(df.sms=='não informado',False).otherwise(True))
df.show()

In [ ]:
#8-Crie uma coluna chamada “total” que deve ser a multiplicação das colunas “vlr” e “qtd”.
df = df.withColumn("total",df.vlr * df.qtd)
df

DataFrame[cod_ean: string, cod_pessoa: decimal(10,4), cod_transacao: decimal(10,2), data: date, vlr: bigint, qtd: bigint, nome: string, sms: boolean, email: boolean, classe: string, sexo: string, cidade: string, des_produto: string, des_familia: string, des_secao: string, des_categoria: string, des_sub_categoria: string, __index_level_0__: bigint, ano: int, mes: int, dia: int, total: bigint]

In [ ]:
df1 = spark.read.option("basePath","/content/ac4/vendas_partition/").parquet("/content/ac4/vendas_partition/ano=2019/mes=10")
df1.select("ano","mes","dia").distinct().show()

In [ ]:
}#9-Grave o dataframe particionado por ano, mes e dia no formato “parquet” de nome “vendas_tratadas”.
#df.write.mode("overwrite").partitionBy("ano").parquet("ac4/vendas_ano")
#df.write.mode("overwrite").partitionBy("mes").parquet("ac4/vendas_mes")
#df.write.mode("overwrite").partitionBy("dia").parquet("ac4/vendas_dia")
df.write.mode("overwrite").partitionBy("ano","mes","dia").parquet("ac4/vendas_partition")

In [ ]:
df2019 = spark.read.parquet("/content/ac4/vendas_partition/ano=2019")

In [ ]:
#10-Crie uma tabela temporaria chamada “vendas_table” (dataFrame.registerTempTable(“vendas_table”))
df.registerTempTable("vendas_table")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [ ]:
#11-Crie um dataframe spark.sql utilizando a query 
#spark.sql(“select distinct cidade from vendas_table”). Grave esse dataframe no formato “parquet” de nome  “cidade”.
df_ci = spark.sql("  SELECT distinct cidade FROM vendas_table    ")
df_ci.registerTempTable("cidade")
df_ci.write.mode("overwrite").parquet("ac4/cidade")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [ ]:
#12-Criar um dataframe com a soma do vlr, soma da qtd agrupado pela data. 
#Gravar esse dataframe no formato “parquet” de nome “vendas_total”. 
df_agg= spark.sql("select sum(vlr) vlr,sum(qtd) qtd, data from vendas_table  group by data ")
df_agg1 = df.groupBy("data").agg(f.sum(df.qtd).alias("qtd"),f.sum(df.vlr).alias("vlr"))
df_agg1.write.parquet("ac4/vendas_total")

In [ ]:
df_sex = df.groupBy("sexo","classe").agg(f.max("data").alias("max_data"),
                                         f.avg("vlr").alias("media_valor")
                                        ,f.min("data").alias("data_minima"))
df_sex.show()

+----+------+----------+------------------+-----------+
|sexo|classe|  max_data|       media_valor|data_minima|
+----+------+----------+------------------+-----------+
|   F|  Ouro|2021-01-16|17.554443554843875| 2019-09-01|
|   M|  Ouro|2021-01-16| 35.07774560035643| 2019-09-01|
|   F| Prata|2021-01-14| 20.11247443762781| 2019-09-10|
|   M| Prata|2021-01-16|  43.0746336996337| 2019-09-02|
|null|  null|2021-01-16| 29.44274077623383| 2019-09-01|
+----+------+----------+------------------+-----------+



In [ ]:
#13-Crie um novo dataframe chamado “produto” selecionando as colunas:
# “des_produto”,”des_familia”, “des_secao”, “des_categoria”,”des_sub_categoria”. Apenas valores unicos (distinct()). Grave esse dataframe como parquet “produto”
df_p = df.select(f.trim("des_produto").alias("des_produto"),
                 f.trim("des_familia").alias("des_familia"),
                 f.trim("des_secao").alias("des_secao"),
                 f.trim("des_categoria").alias("des_categoria"),
                 f.trim("des_sub_categoria").alias("des_sub_categoria")).distinct()
df_p = df_p.where(df_p.des_produto.isNotNull())
df_p = df_p.drop("des_sub_categoria")
df_p.orderBy("des_produto").show()
df_p.write.partitionBy("des_familia","des_secao").parquet("ac4/produto")

+--------------------+-----------+---------+-------------------+
|         des_produto|des_familia|des_secao|      des_categoria|
+--------------------+-----------+---------+-------------------+
| ACEM BOVINO KG PECA| PERECIVEIS|  ACOUGUE|       CARNE BOVINA|
|ACUCAR CRISTAL SA...|  MERCEARIA|ALTO GIRO|             ACUCAR|
|ALCATRA C/ MAMINH...| PERECIVEIS|  ACOUGUE|       CARNE BOVINA|
|   ALHO GRANEL T6 KG| PERECIVEIS|    F L V|               ALHO|
|ARROZ EMPORIO SAO...|  MERCEARIA|ALTO GIRO|              ARROZ|
|ARROZ SANO 5 KG T...|  MERCEARIA|ALTO GIRO|              ARROZ|
|    BANANA NANICA KG| PERECIVEIS|    F L V|             FRUTAS|
|     BATATA EXTRA KG| PERECIVEIS|    F L V|TUBERCULOS E RAIZES|
|  CEBOLA NACIONAL KG| PERECIVEIS|    F L V|             CEBOLA|
|          CENOURA KG| PERECIVEIS|    F L V|TUBERCULOS E RAIZES|
|CERVEJA AMSTEL 35...|  MERCEARIA|  BEBIDAS|           CERVEJAS|
|CERVEJA SKOL 350 ...|  MERCEARIA|  BEBIDAS|           CERVEJAS|
|CONTRA FILE BOVIN...| PE

In [ ]:
#14-Crie um dataframe chamado “venda_mulheres”, selecionando todas as colunas e filtrando apenas vendas realizadas para o sexo “F”,
# grave esse dataframe no formato json “venda_mulher”. (df.write.format(“json”).save(“venda_mulher”))
df_f = spark.sql("select * from vendas_table where sexo = 'F'")
df_f1 = df.where(df.sexo=="F")
df_f1 = df.filter("sexo =  'F'  ")
df_f1.write.json("ac4/venda_mulher")

In [ ]:
#15 #15-Crie um dataframe chamado “venda_unicas”, selecionando todas as colunas e filtrando apenas vendas com a qtd igual 1 
#e o vlr menor que 10. Grave esse dataframe no formato csv“venda_unica”. (df.write.format(“csv”).save(“venda_unica”))
dfu = df.where((df.qtd==1)&(df.vlr<10))
dfu.write.csv("ac4/venda_unica")

In [ ]:
dfc = spark.read.option("header","true").option("delimiter",";").csv("/content/cliente.csv")
dfc1 = spark.read.option("header","true").option("delimiter",";").csv("/content/cliente (1).csv")
dfn = spark.read.option("header","true").option("delimiter",";").csv("/content/nota.csv")

In [ ]:
dfc.show()

+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+
|id_pessoa|            nome|   rg|      cpf|id_sexo|id_estado_civil|             email| telefone|id_endereco|
+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+
|        1|   felipe carlos|11064|343440908|      1|              1|      wh!email.net|1122-1111|          4|
|        2|    eduardo lins| 9885|  4563019|      1|              1| whsssss@email.net|1122-1112|          5|
|        3|   alexandre mar|10085| 56652019|      2|              1|  eeeeeh@email.net|1122-1113|          6|
|        4|   Julio casares|33297|  7763019|      1|              1|eeeeeewh@email.net|1122-1114|          7|
|        5|    mirande lean|14307|  4553140|      1|              2|  eeeewh@email.net|1122-1115|          8|
|        6|   peter sanders|16418|  3440908|      1|              2|              null|1122-1116|          9|
|        7

In [ ]:
dfc1.show()

+----------+--------------+-----+---------+-------+---------------+------------------+---------+-----------+
|id_cliente|          nome|   rg|      cpf|id_sexo|id_estado_civil|             email| telefone|id_endereco|
+----------+--------------+-----+---------+-------+---------------+------------------+---------+-----------+
|         1|  jose armando| 1211|343434343|      1|              1|      yh@email.com|2222-1111|          4|
|         2| carlo eduardo|   32|  4556454|      1|              1| yhsssss@email.com|2222-1112|          5|
|         3|    nina lopes|  232| 56645454|      2|              1|  eeeeeh@email.com|2222-1113|          6|
|         4|   juan arturo|23444|  7756454|      1|              1|eeeeddyh@email.com|2222-1115|          7|
|         5|micheal marley| 4454|  4546575|      1|              2|  ddddyh@email.com|2222-1111|          8|
|         6|        jhonny| 6565|  3434343|      1|              2|              ssss|2122-1111|          9|
|         7| carlo 

In [ ]:
dfu = dfc.union(dfc1)
dfu.show()

+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+
|id_pessoa|            nome|   rg|      cpf|id_sexo|id_estado_civil|             email| telefone|id_endereco|
+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+
|        1|   felipe carlos|11064|343440908|      1|              1|      wh!email.net|1122-1111|          4|
|        2|    eduardo lins| 9885|  4563019|      1|              1| whsssss@email.net|1122-1112|          5|
|        3|   alexandre mar|10085| 56652019|      2|              1|  eeeeeh@email.net|1122-1113|          6|
|        4|   Julio casares|33297|  7763019|      1|              1|eeeeeewh@email.net|1122-1114|          7|
|        5|    mirande lean|14307|  4553140|      1|              2|  eeeewh@email.net|1122-1115|          8|
|        6|   peter sanders|16418|  3440908|      1|              2|              null|1122-1116|          9|
|        7

In [ ]:
dfn.show()

+-------+---------+----------+--------------+
|id_nota|id_pessoa|      data|id_consultorio|
+-------+---------+----------+--------------+
|      1|        1|01/01/2020|             1|
|      2|        2|01/01/2020|             2|
|      3|        3|01/01/2020|             3|
|      4|        4|02/01/2020|             1|
|      5|        5|02/01/2020|             2|
|      6|        6|02/01/2020|             3|
|      7|        7|02/01/2020|             1|
|      8|        8|01/02/2020|             2|
|      9|        9|01/02/2020|             3|
|     10|       10|01/02/2020|             1|
|     11|        1|01/02/2020|             2|
|     12|        2|01/02/2020|             3|
|     13|        3|01/02/2020|             1|
|     14|        4|01/02/2020|             2|
|     15|        5|01/02/2020|             3|
|     16|        6|01/02/2020|             1|
|     17|        7|03/03/2020|             2|
|     18|        8|03/03/2020|             3|
|     19|        9|03/03/2020|    

In [ ]:
dfj = dfc.join(dfn,(dfc.id_pessoa==dfn.id_pessoa),how='left')
dfj = dfc.join(dfn,"id_pessoa",how='inner')
dfj.show()

+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+-------+----------+--------------+
|id_pessoa|            nome|   rg|      cpf|id_sexo|id_estado_civil|             email| telefone|id_endereco|id_nota|      data|id_consultorio|
+---------+----------------+-----+---------+-------+---------------+------------------+---------+-----------+-------+----------+--------------+
|        1|   felipe carlos|11064|343440908|      1|              1|      wh!email.net|1122-1111|          4|      1|01/01/2020|             1|
|        2|    eduardo lins| 9885|  4563019|      1|              1| whsssss@email.net|1122-1112|          5|      2|01/01/2020|             2|
|        3|   alexandre mar|10085| 56652019|      2|              1|  eeeeeh@email.net|1122-1113|          6|      3|01/01/2020|             3|
|        4|   Julio casares|33297|  7763019|      1|              1|eeeeeewh@email.net|1122-1114|          7|      4|02/01/2020|        